# Advanced Level: Machine Learning & Predictive Modeling for DEI Analysis

Welcome to the advanced workshop on DEI in the music industry! This notebook covers sophisticated analytical techniques including machine learning, predictive modeling, and advanced statistical methods.

## Learning Objectives:
- Build machine learning models to predict artist success
- Identify bias and discrimination patterns using ML techniques
- Perform dimensionality reduction and clustering analysis
- Apply advanced feature engineering
- Conduct bias detection and fairness analysis
- Create recommendation systems for equitable representation

## Prerequisites:
You should have completed both beginner and intermediate levels, or have solid experience with statistics, pandas, and basic machine learning concepts.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.inspection import permutation_importance

# Statistical libraries
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("All libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('../data/music_industry_dei_data.csv')

print(f"Dataset loaded: {df.shape[0]} artists, {df.shape[1]} features")
print(f"Features: {list(df.columns)}")
df.head()

## Feature Engineering for ML Models

Let's create sophisticated features that capture complex patterns in the data.

In [ ]:
# Create advanced features
def engineer_features(df):
    """Create advanced features for machine learning"""
    df = df.copy()
    
    # Success metrics per year
    df['listeners_per_year'] = df['monthly_listeners'] / df['years_active']
    df['streams_per_year'] = df['total_streams'] / df['years_active']
    df['albums_per_year'] = df['album_count'] / df['years_active']
    df['awards_per_year'] = df['award_wins'] / df['years_active']
    
    # Platform ratio features
    df['spotify_soundcloud_ratio'] = df['spotify_followers'] / (df['soundcloud_followers'] + 1)
    df['followers_to_listeners_ratio'] = df['spotify_followers'] / (df['monthly_listeners'] + 1)
    
    # Success intensity (streams per listener)
    df['streams_per_listener'] = df['total_streams'] / (df['monthly_listeners'] + 1)
    
    # Create interaction features
    df['gender_ethnicity'] = df['gender'] + '_' + df['ethnicity']
    df['genre_label'] = df['genre'] + '_' + df['label_type']
    
    # Categorical encoding
    le_gender = LabelEncoder()
    le_ethnicity = LabelEncoder()
    le_genre = LabelEncoder()
    le_country = LabelEncoder()
    le_label = LabelEncoder()
    
    df['gender_encoded'] = le_gender.fit_transform(df['gender'])
    df['ethnicity_encoded'] = le_ethnicity.fit_transform(df['ethnicity'])
    df['genre_encoded'] = le_genre.fit_transform(df['genre'])
    df['country_encoded'] = le_country.fit_transform(df['country'])
    df['label_encoded'] = le_label.fit_transform(df['label_type'])
    
    # Success tier (categorical target for classification)
    df['success_tier'] = pd.cut(df['monthly_listeners'], 
                               bins=[0, 5000000, 25000000, 50000000, np.inf],
                               labels=['Emerging', 'Rising', 'Mainstream', 'Superstar'])
    
    return df, le_gender, le_ethnicity, le_genre, le_country, le_label

df_engineered, le_gender, le_ethnicity, le_genre, le_country, le_label = engineer_features(df)

print(f"Engineered dataset shape: {df_engineered.shape}")
print(f"New features created: {set(df_engineered.columns) - set(df.columns)}")
df_engineered.head()

## Predictive Modeling: Success Prediction

Let's build machine learning models to predict artist success and identify important factors.

In [ ]:
# Prepare features for modeling
feature_cols = ['years_active', 'album_count', 'gender_encoded', 'ethnicity_encoded', 
               'genre_encoded', 'country_encoded', 'label_encoded', 'award_wins',
               'listeners_per_year', 'albums_per_year', 'awards_per_year',
               'spotify_soundcloud_ratio']

X = df_engineered[feature_cols]
y_listeners = df_engineered['monthly_listeners']
y_streams = df_engineered['total_streams']

# Split the data
X_train, X_test, y_train_listeners, y_test_listeners = train_test_split(
    X, y_listeners, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {feature_cols}")

In [ ]:
# Train multiple models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

model_results = {}

for name, model in models.items():
    # Use scaled data for linear models, original for tree-based models
    if name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression']:
        X_train_use, X_test_use = X_train_scaled, X_test_scaled
    else:
        X_train_use, X_test_use = X_train, X_test
    
    # Train model
    model.fit(X_train_use, y_train_listeners)
    
    # Make predictions
    y_pred = model.predict(X_test_use)
    
    # Calculate metrics
    mse = mean_squared_error(y_test_listeners, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_listeners, y_pred)
    r2 = r2_score(y_test_listeners, y_pred)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_use, y_train_listeners, cv=5, scoring='r2')
    
    model_results[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'CV_R²_mean': cv_scores.mean(),
        'CV_R²_std': cv_scores.std(),
        'model': model,
        'predictions': y_pred
    }

# Display results
results_df = pd.DataFrame({k: {metric: v[metric] for metric in ['RMSE', 'MAE', 'R²', 'CV_R²_mean', 'CV_R²_std']} 
                          for k, v in model_results.items()}).T

print("Model Performance Comparison:")
print(results_df.round(4))

# Find best model
best_model_name = results_df['R²'].idxmax()
print(f"\nBest performing model: {best_model_name} (R² = {results_df.loc[best_model_name, 'R²']:.4f})")

## Feature Importance and Bias Detection

Let's analyze which features are most important for predicting success and identify potential bias.

In [ ]:
# Get the best model
best_model = model_results[best_model_name]['model']

# Feature importance analysis
if hasattr(best_model, 'feature_importances_'):
    feature_importance = best_model.feature_importances_
else:
    # For linear models, use coefficient magnitudes
    feature_importance = np.abs(best_model.coef_)

# Create feature importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(data=importance_df, x='importance', y='feature')
plt.title(f'Feature Importance - {best_model_name}')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print("Feature Importance Ranking:")
print(importance_df)

In [ ]:
# Bias detection: Analyze prediction errors by demographic groups
test_results = X_test.copy()
test_results['actual'] = y_test_listeners
test_results['predicted'] = model_results[best_model_name]['predictions']
test_results['error'] = test_results['actual'] - test_results['predicted']
test_results['abs_error'] = np.abs(test_results['error'])
test_results['pct_error'] = (test_results['error'] / test_results['actual']) * 100

# Map encoded values back to original categories
test_results['gender'] = le_gender.inverse_transform(test_results['gender_encoded'])
test_results['ethnicity'] = le_ethnicity.inverse_transform(test_results['ethnicity_encoded'])
test_results['genre'] = le_genre.inverse_transform(test_results['genre_encoded'])
test_results['label_type'] = le_label.inverse_transform(test_results['label_encoded'])

# Analyze bias by gender
gender_bias = test_results.groupby('gender').agg({
    'error': ['mean', 'std'],
    'abs_error': 'mean',
    'pct_error': 'mean'
}).round(2)

print("BIAS ANALYSIS BY GENDER:")
print(gender_bias)

# Analyze bias by ethnicity
ethnicity_bias = test_results.groupby('ethnicity').agg({
    'error': ['mean', 'std'],
    'abs_error': 'mean',
    'pct_error': 'mean'
}).round(2)

print("\nBIAS ANALYSIS BY ETHNICITY:")
print(ethnicity_bias)

In [ ]:
# Visualize prediction bias
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Actual vs Predicted by Gender
for i, gender in enumerate(test_results['gender'].unique()):
    gender_data = test_results[test_results['gender'] == gender]
    axes[0,0].scatter(gender_data['actual'], gender_data['predicted'], 
                     label=gender, alpha=0.7)
axes[0,0].plot([0, test_results['actual'].max()], [0, test_results['actual'].max()], 
              'k--', label='Perfect Prediction')
axes[0,0].set_xlabel('Actual Monthly Listeners')
axes[0,0].set_ylabel('Predicted Monthly Listeners')
axes[0,0].set_title('Actual vs Predicted by Gender')
axes[0,0].legend()

# Error distribution by Gender
sns.boxplot(data=test_results, x='gender', y='pct_error', ax=axes[0,1])
axes[0,1].set_title('Prediction Error Distribution by Gender')
axes[0,1].axhline(y=0, color='red', linestyle='--', alpha=0.7)

# Error by Ethnicity
sns.boxplot(data=test_results, x='ethnicity', y='pct_error', ax=axes[1,0])
axes[1,0].set_title('Prediction Error Distribution by Ethnicity')
axes[1,0].set_xticklabels(axes[1,0].get_xticklabels(), rotation=45)
axes[1,0].axhline(y=0, color='red', linestyle='--', alpha=0.7)

# Residuals plot
axes[1,1].scatter(test_results['predicted'], test_results['error'], alpha=0.7)
axes[1,1].axhline(y=0, color='red', linestyle='--')
axes[1,1].set_xlabel('Predicted Values')
axes[1,1].set_ylabel('Residuals')
axes[1,1].set_title('Residuals vs Predicted Values')

plt.tight_layout()
plt.show()

## Clustering Analysis: Artist Archetypes

Let's use unsupervised learning to identify different artist archetypes and analyze their demographic composition.

In [ ]:
# Prepare data for clustering
cluster_features = ['monthly_listeners', 'total_streams', 'album_count', 'years_active', 
                   'award_wins', 'spotify_followers', 'soundcloud_followers']

X_cluster = df_engineered[cluster_features]

# Scale the data
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

# Determine optimal number of clusters using elbow method
inertias = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_cluster_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertias, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()

# Choose optimal k (look for the elbow)
optimal_k = 4  # You can adjust this based on the elbow curve
print(f"Using k = {optimal_k} clusters")

In [ ]:
# Perform clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(X_cluster_scaled)

# Add cluster labels to dataframe
df_clustered = df_engineered.copy()
df_clustered['cluster'] = clusters

# Analyze cluster characteristics
cluster_summary = df_clustered.groupby('cluster')[cluster_features].mean()
print("Cluster Characteristics (Mean Values):")
print(cluster_summary.round(0))

# Analyze demographic composition of clusters
print("\nCluster Demographic Composition:")
for cluster in range(optimal_k):
    cluster_data = df_clustered[df_clustered['cluster'] == cluster]
    print(f"\nCluster {cluster} ({len(cluster_data)} artists):")
    print(f"  Gender: {cluster_data['gender'].value_counts().to_dict()}")
    print(f"  Ethnicity: {cluster_data['ethnicity'].value_counts().to_dict()}")
    print(f"  Top Genres: {cluster_data['genre'].value_counts().head(3).to_dict()}")

In [ ]:
# Visualize clusters using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_cluster_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.7)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.title('Artist Clusters in PCA Space')
plt.colorbar(scatter)

# Add cluster centers
centers_pca = pca.transform(kmeans.cluster_centers_)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], c='red', marker='x', s=300, linewidths=3)

plt.show()

print(f"PCA explains {pca.explained_variance_ratio_.sum():.2%} of total variance")

## Fairness Metrics and Bias Quantification

Let's implement formal fairness metrics to quantify bias in artist success patterns.

In [ ]:
# Define fairness metrics
def calculate_fairness_metrics(df, protected_attribute, outcome_variable, threshold_percentile=75):
    """
    Calculate various fairness metrics for a protected attribute
    """
    threshold = np.percentile(df[outcome_variable], threshold_percentile)
    df['high_success'] = (df[outcome_variable] >= threshold).astype(int)
    
    metrics = {}
    groups = df[protected_attribute].unique()
    
    # Statistical Parity (Demographic Parity)
    group_rates = {}
    for group in groups:
        group_data = df[df[protected_attribute] == group]
        positive_rate = group_data['high_success'].mean()
        group_rates[group] = positive_rate
    
    # Calculate max difference in positive rates
    max_diff = max(group_rates.values()) - min(group_rates.values())
    metrics['statistical_parity_diff'] = max_diff
    
    # Equalized Odds (True Positive Rate equality)
    # For this context, we'll use average success rate as proxy
    avg_success_by_group = df.groupby(protected_attribute)[outcome_variable].mean()
    success_ratio = avg_success_by_group.max() / avg_success_by_group.min()
    metrics['success_ratio'] = success_ratio
    
    # Representation in top tier
    top_tier = df[df['high_success'] == 1]
    top_tier_composition = top_tier[protected_attribute].value_counts(normalize=True)
    overall_composition = df[protected_attribute].value_counts(normalize=True)
    
    representation_ratio = {}
    for group in groups:
        top_pct = top_tier_composition.get(group, 0)
        overall_pct = overall_composition.get(group, 0)
        if overall_pct > 0:
            representation_ratio[group] = top_pct / overall_pct
        else:
            representation_ratio[group] = 0
    
    metrics['group_rates'] = group_rates
    metrics['avg_success_by_group'] = avg_success_by_group.to_dict()
    metrics['representation_ratio'] = representation_ratio
    
    return metrics

# Calculate fairness metrics for gender
gender_fairness = calculate_fairness_metrics(df_engineered, 'gender', 'monthly_listeners')

print("FAIRNESS ANALYSIS - GENDER:")
print(f"Statistical Parity Difference: {gender_fairness['statistical_parity_diff']:.3f}")
print(f"Success Ratio (Max/Min): {gender_fairness['success_ratio']:.3f}")
print(f"High Success Rates by Gender: {gender_fairness['group_rates']}")
print(f"Average Success by Gender: {gender_fairness['avg_success_by_group']}")
print(f"Representation Ratio in Top Tier: {gender_fairness['representation_ratio']}")

print("\n" + "="*60 + "\n")

# Calculate fairness metrics for ethnicity
ethnicity_fairness = calculate_fairness_metrics(df_engineered, 'ethnicity', 'monthly_listeners')

print("FAIRNESS ANALYSIS - ETHNICITY:")
print(f"Statistical Parity Difference: {ethnicity_fairness['statistical_parity_diff']:.3f}")
print(f"Success Ratio (Max/Min): {ethnicity_fairness['success_ratio']:.3f}")
print(f"High Success Rates by Ethnicity:")
for eth, rate in ethnicity_fairness['group_rates'].items():
    print(f"  {eth}: {rate:.3f}")
print(f"Representation Ratio in Top Tier:")
for eth, ratio in ethnicity_fairness['representation_ratio'].items():
    print(f"  {eth}: {ratio:.3f} ({'Over' if ratio > 1 else 'Under'}-represented)")

## Recommendation System for Equitable Representation

Let's create a simple recommendation system to identify underrepresented artists who deserve more attention.

In [ ]:
def create_equity_score(df):
    """
    Create an equity score that identifies underrepresented artists with high potential
    """
    df = df.copy()
    
    # Quality metrics (normalized to 0-1)
    df['quality_score'] = (
        (df['award_wins'] / df['award_wins'].max()) * 0.3 +
        (df['albums_per_year'] / df['albums_per_year'].max()) * 0.2 +
        (df['years_active'] / df['years_active'].max()) * 0.2 +
        (df['streams_per_listener'] / df['streams_per_listener'].max()) * 0.3
    )
    
    # Current visibility (inverse - lower visibility = higher equity need)
    df['visibility_score'] = 1 - (df['monthly_listeners'] / df['monthly_listeners'].max())
    
    # Demographic penalty (higher for underrepresented groups)
    demographic_weights = {
        'Female': 1.3,  # Female artists are underrepresented at top levels
        'Male': 1.0
    }
    
    ethnicity_weights = {
        'White': 1.0,
        'Black': 1.2,
        'Latino': 1.2,
        'Asian': 1.3,
        'Mixed': 1.2,
        'Middle Eastern': 1.4
    }
    
    df['demo_weight'] = (df['gender'].map(demographic_weights) * 
                        df['ethnicity'].map(ethnicity_weights))
    
    # Independent label boost
    df['label_boost'] = df['label_type'].map({'Independent': 1.2, 'Major': 1.0})
    
    # Calculate final equity score
    df['equity_score'] = (df['quality_score'] * 0.4 + 
                         df['visibility_score'] * 0.3 + 
                         df['demo_weight'] * 0.2 + 
                         df['label_boost'] * 0.1)
    
    return df

df_equity = create_equity_score(df_engineered)

# Top recommendations for equitable promotion
top_equity = df_equity.nlargest(15, 'equity_score')[[
    'artist_name', 'gender', 'ethnicity', 'genre', 'country', 'label_type',
    'monthly_listeners', 'quality_score', 'visibility_score', 'equity_score'
]].round(3)

print("TOP 15 ARTISTS FOR EQUITABLE PROMOTION:")
print(top_equity.to_string(index=False))

In [ ]:
# Visualize equity scores
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=['Equity Score by Gender', 'Equity Score by Ethnicity',
                                  'Quality vs Visibility', 'Equity Score Distribution'],
                    specs=[[{"type": "box"}, {"type": "box"}],
                          [{"type": "scatter"}, {"type": "histogram"}]])

# Box plot by gender
for gender in df_equity['gender'].unique():
    gender_data = df_equity[df_equity['gender'] == gender]['equity_score']
    fig.add_trace(go.Box(y=gender_data, name=gender, showlegend=False), row=1, col=1)

# Box plot by ethnicity
for ethnicity in df_equity['ethnicity'].unique():
    eth_data = df_equity[df_equity['ethnicity'] == ethnicity]['equity_score']
    fig.add_trace(go.Box(y=eth_data, name=ethnicity, showlegend=False), row=1, col=2)

# Scatter: Quality vs Visibility
fig.add_trace(go.Scatter(x=df_equity['quality_score'], y=df_equity['visibility_score'],
                        mode='markers', 
                        marker=dict(color=df_equity['equity_score'], colorscale='Viridis',
                                  showscale=True, colorbar=dict(title="Equity Score")),
                        text=df_equity['artist_name'],
                        showlegend=False), row=2, col=1)

# Histogram of equity scores
fig.add_trace(go.Histogram(x=df_equity['equity_score'], nbinsx=20, showlegend=False), row=2, col=2)

fig.update_layout(height=800, title_text="Equity Analysis Dashboard")
fig.update_xaxes(title_text="Quality Score", row=2, col=1)
fig.update_yaxes(title_text="Visibility Score", row=2, col=1)
fig.update_xaxes(title_text="Equity Score", row=2, col=2)
fig.update_yaxes(title_text="Count", row=2, col=2)

fig.show()

## Advanced Insights and Recommendations

Based on your advanced analysis, provide comprehensive insights:

### Your Advanced Insights:

1. **Machine Learning Model Performance**: [Analyze which model performed best and what this tells us about the predictability of success]

2. **Feature Importance**: [Discuss the most important factors for predicting success and their implications for DEI]

3. **Bias Detection**: [Analyze the systematic biases found in the model predictions and their real-world implications]

4. **Artist Archetypes**: [Describe the different clusters/archetypes found and their demographic composition]

5. **Fairness Metrics**: [Interpret the fairness metrics and what they reveal about equity in the music industry]

6. **Equity Recommendations**: [Discuss the recommended artists and the rationale behind the equity scoring system]

7. **Systemic Issues**: [Identify systemic issues that contribute to inequality in the music industry]

8. **Actionable Solutions**: [Propose specific, data-driven solutions for improving DEI in the music industry]

## Technical Limitations and Future Work

### Limitations of This Analysis:
- Small sample size may not represent the full music industry
- Success metrics are limited to streaming/followers (missing radio play, live performance revenue, etc.)
- Temporal dynamics not captured (how has representation changed over time?)
- Selection bias: only includes already successful artists
- Intersectionality not fully explored (e.g., gender × ethnicity interactions)

### Future Research Directions:
- Longitudinal analysis to track changes over time
- Causal inference methods to identify causal factors vs correlations
- Natural language processing on lyrics and music reviews
- Network analysis of collaborations and industry connections
- Integration of additional data sources (radio play, concert attendance, etc.)
- Deeper intersectional analysis

## Conclusion

Congratulations! You've completed an advanced analysis using:

- **Machine Learning**: Predictive modeling with multiple algorithms
- **Bias Detection**: Systematic analysis of algorithmic bias
- **Clustering**: Unsupervised learning to identify artist archetypes
- **Fairness Metrics**: Quantitative assessment of equity
- **Recommendation Systems**: Data-driven approach to promoting equity
- **Advanced Visualization**: Interactive and multi-dimensional plots

This analysis demonstrates how advanced data science techniques can be applied to understand and address DEI challenges in creative industries.